In [3]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다"""

- 토큰화

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

t = Tokenizer()
# 문장 분석
t.fit_on_texts([text])

# 분석된 단어의 목록을 확인
t.word_docs

defaultdict(int,
            {'법이다': 1,
             '말이': 1,
             '있다': 1,
             '오는': 1,
             '뛰고': 1,
             '있는': 1,
             '그의': 1,
             '고와야': 1,
             '가는': 1,
             '곱다': 1,
             '경마장에': 1})

In [5]:
# 단어의 수를 저장
vocab_size = len(t.word_docs) + 1

vocab_size

12

- 정수 인코딩

In [6]:
seq = []

# 문장을 한 줄씩 읽어온다
for line in text.split("\n") :
  encoded = t.texts_to_sequences([line])[0]

  # 조합 가능한 단어 조합을 생성
  for i in range(1, len(encoded)) :
    s = encoded[:i+1]
    seq.append(s)

seq    

# 경마장에 있는 말이 뛰고 있다
#    경마장에 있는
#    경마장에 있는 말이
#    경마장에 있는 말이 뛰고
#    경마장에 있는 말이 뛰고 있다
# 그의 말이 법이다
#    그의 말이
#    그의 말이 법이다
# 가는 말이 고와야 오는 말이 곱다
#     가는 말이
#     가는 말이 고와야
#     가는 말이 고와야 오는
#     가는 말이 고와야 오는 말이
#     가는 말이 고와야 오는 말이 곱다

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

- 인코딩된 결과를 같은 길이로 만듬

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 최대 길이의 문자의 단어수
max_len = 6

sequence = pad_sequences(seq, maxlen=max_len)
sequence

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]], dtype=int32)

- 특성데이터와 라벨데이터로 분리

In [8]:
X = sequence[:, :-1]
y = sequence[:, -1]

X.shape, y.shape

((11, 5), (11,))

- y를 원핫인코딩

In [9]:
# 값이 수치 형태의 데이터인 경우에만 원핫인코딩
from tensorflow.keras.utils import to_categorical

y_en = to_categorical(y, num_classes=vocab_size)

y_en.shape

(11, 12)

In [10]:
y_en

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [11]:
y_en.shape

(11, 12)

- 신경망 설계

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

model1 = Sequential()

model1.add(Embedding(vocab_size, 10, input_length=5))

model1.add(SimpleRNN(32))

model1.add(Dense(12, activation="softmax"))

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 10)             120       
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1376      
                                                                 
 dense (Dense)               (None, 12)                396       
                                                                 
Total params: 1,892
Trainable params: 1,892
Non-trainable params: 0
_________________________________________________________________


In [13]:
model1.compile(loss="categorical_crossentropy",
               optimizer="adam",
               metrics=["accuracy"])

In [14]:
h1 = model1.fit(X, y_en, epochs=200)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 2.4886 - accuracy: 0.0909
Epoch 2/200
1/1 [==============================] - 0s 12ms/step - loss: 2.4756 - accuracy: 0.0909
Epoch 3/200
1/1 [==============================] - 0s 9ms/step - loss: 2.4624 - accuracy: 0.0909
Epoch 4/200
1/1 [==============================] - 0s 11ms/step - loss: 2.4489 - accuracy: 0.0909
Epoch 5/200
1/1 [==============================] - 0s 8ms/step - loss: 2.4351 - accuracy: 0.0909
Epoch 6/200
1/1 [==============================] - 0s 8ms/step - loss: 2.4208 - accuracy: 0.1818
Epoch 7/200
1/1 [==============================] - 0s 9ms/step - loss: 2.4060 - accuracy: 0.1818
Epoch 8/200
1/1 [==============================] - 0s 8ms/step - loss: 2.3905 - accuracy: 0.1818
Epoch 9/200
1/1 [==============================] - 0s 7ms/step - loss: 2.3743 - accuracy: 0.2727
Epoch 10/200
1/1 [==============================] - 0s 12ms/step - loss: 2.3573 - accuracy: 0.3636
Epoch 11/200
1/1 [=========

- 문장을 생성하는 함수

In [23]:
import numpy as np

# model : 사용한 모델
# t : 사용한 Tonkenizer
# word : 입력단어
# n : 입력단어로부터 예측할 단어 수
def generate_seqence(model, t, current_word, n) :
  init_word = current_word  # 입력된 단어로 시작해야 하므로 입력단어를 먼저 저장

  # 생성한 문장을 저장할 변수 초기화
  seq = ""

  for _ in range(n) :
    # 현재 단어를 인코딩하고 padding 처리를 수행
    encoded = t.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=max_len-1)

    # 현재 단어로 다음 단어를 예측 (인덱스 반환)
    result = np.argmax(model.predict(encoded))

    #print(result)  
    for word, index in t.word_index.items() :
      # 예측한 단어와 동일한 인덱스의 단어가 있다면 종료
      if index == result :
        break
      
    # 현재단어와 예측한 단어를 연결
    current_word = current_word + " " + word

    seq = seq + " " + word

  seq = init_word + seq

  return seq 

In [31]:
generate_seqence(model1, t, "오는 말이", 2)

1/1 [==============================] - 0s 15ms/step


'오는 말이 고와야 오는'